In [ ]:
%pip install langchain llama-cpp-python 
#CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

### Load Packages

In [76]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import os

### Define System Prompt and load Llama2 Model

In [77]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
# DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information"

In [78]:
#Chatbot prompt for later use case
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are a chatbot that helps people learn a new language. You always give the answer to the question in a formal manner.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT +E_SYS

In [79]:
#general prompt to test llm for testing questions
SYSTEM_PROMPT_2 = """
<<SYS>>
You are alwys giving the answer to the question in a short and very simple manner and you answer in the language style that is asked from you. You leave out unnecessary words and if you have several options you would like to present only name one. If the answer is only one word you don't put a dot and the end. If you are asked to translate something you translate it.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT_2 +E_SYS

In [80]:
def get_prompt(instruction):
    return B_INST + SYSTEM_PROMPT + instruction + E_INST

os.listdir("/Users/josi/Llama2_weights")

['llama-7b.ggmlv3.q4_K_M.bin',
 '.DS_Store',
 'em_german_leo_mistral.Q5_K_M.gguf',
 'llama-2-7b.Q4_K_M.gguf',
 'llama-2-7b-chat.Q5_K_M.gguf']

In [87]:
llm = LlamaCpp(
    model_path= "/Users/josi/Llama2_weights/llama-2-7b.Q4_K_M.gguf",
    #model_path = "/Users/josi/Llama2_weights/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=2048,
    top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/josi/Llama2_weights/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,  

### Create Dictionaries with test questions (English)

In [88]:
# Define test questions for different categories
informal_questions = {
    "Write an informal greeting.": "Hey",
    "Write an informal response to the question: 'How's it going?'": "Hi, going good, hbu?",
    "Write an informal message inviting someone to dinner tonight": "Hey, wanna join us for dinner tonight?",
    "Compose an informal email closing.": "Cheers, [Your Name]",
}

formal_questions = {
    "Write a formal greeting.": "Hello",
    "Write a formal response to the question: 'How is it going?'": "Hello, I am doing well. How about you?",
    "Write a formal message inviting someone to dinner tonight": "Hello, would you like to join us for dinner tonight?",
    "Compose a professional email closing.": "Sincerely, [Your Name]",
}

academic_questions = {
    "Describe the french revolution in two sentences using academic language.": "The historical event of the French Revolution was marked by widespread social upheaval...",
    "Define gravity in two sentences using academic language.": "Gravity is a force that attracts two objects with mass",
}

translation_questions = {
    "Translate 'hello' to Spanish.": "Hola",
    "Translate 'good morning' to French.": "Bonjour",
    "Translate 'Thank you' to German.": "Danke",
    "How do you say 'hello' in Japanese?": "Konnichiwa (こんにちは)",
    "Provide the Italian translation for 'book'.": "Libro",
}

general_knowledge_questions = {
    "What is the capital of France?": "The capital of France is Paris",
    "What is the square root of 25?": "The square root of 25 is 5",
    "What is the circumference of the earth?":"The circumference of the earth is approximately 40,075 kilometers",
    "question": "Explain the concept of photosynthesis.", "answer": "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll.",
}

textual_entailment = {
    "question": "Given the statement 'The sun rises in the east,' is the statement 'The earth is flat' likely true or false?", "answer": "False",
    "question": "If 'A implies B' and 'B is false,' what can you conclude about A?", "answer": "Nothing definitive can be concluded about A.",
}

ambiguity_handling = {
    "question": "Provide two different interpretations of the phrase 'bank account.'", "answer": "1. An account with a financial institution. 2. The land alongside a river.",
    "question": "What could the word 'bat' mean in the context of baseball and in the context of an animal?", "answer": "In baseball, a 'bat' is a piece of equipment used to hit the ball. In the context of an animal, a 'bat' is a flying mammal.",
}

reasoning_problem_solving = {
    "question": "If a car travels at 60 miles per hour, how long will it take to cover 120 miles?", "answer": "2 hours",
    "question": "Solve the following math problem: 3x + 5 = 20.", "answer": "x = 5",
}

analogical_reasoning = {
    "question": "If 'cat' is to 'kitten,' what is 'dog' to?", "answer": "'Dog' is to 'puppy'",
    "question": "Identify the relationship between 'pen' and 'ink' and apply the same relationship to 'keyboard.'", "answer": "'Keyboard' is to 'electricity'",
}

commonsense_reasoning = {
    "question": "What might be a common reaction to receiving a gift?", "answer": "A common reaction to receiving a gift is expressing gratitude and appreciation.",
    "question": "Predict a possible consequence of leaving food out in the sun for too long.", "answer": "Leaving food out in the sun for too long may lead to spoilage and bacterial growth.",
}

# Named Entity Recognition (NER)
ner = {
    "question": "Identify the named entities in the sentence 'Elon Musk is the CEO of SpaceX.'", "answer": "Named entities: Elon Musk, SpaceX.",
    "question": "Extract the dates mentioned in the text 'The conference is scheduled for January 25-27, 2023.'", "answer": "Dates: January 25-27, 2023.",
}

# Combine all test questions into a single dictionary
test_questions = {**informal_questions, **formal_questions, **academic_questions, **translation_questions, **general_knowledge_questions,**textual_entailment,**ambiguity_handling,**reasoning_problem_solving,**analogical_reasoning,**commonsense_reasoning,**ner}


### Evaluate Model based on Accuracy

In [ ]:
model_answers = []
def evaluate_model(model, questions):
    correct_answers = 0
    total_questions = len(questions)

    for question, expected_answer in questions.items():
        # Get the model's answer for the current question
        prompt = get_prompt(question)
        model_answer = model(prompt)

        # Remove trailing dots from the model's answer and Compare the model's answer to the expected answer (case-insensitive, whitespace-insensitive)
        model_answer = model_answer.rstrip('.')
        if model_answer.strip().lower() == expected_answer.strip().lower():
            correct_answers += 1

        # Check if any of the expected words are present in the model's answer
        #if any(word.lower() in model_answer.lower() for word in expected_answer.split()):
        #    correct_answers += 1

        # Print the results for each question
        print(f"Question: {question}")
        print(f"Expected Answer: {expected_answer}")
        print(f"Model Answer: {model_answer}")
        print("------------")

        # Store the model's answer and the corresponding question in the array
        model_answers.append({
            "question": question,
            "expected_answer": expected_answer,
            "model_answer": model_answer,
        })

    # Print overall evaluation results
    print(f"Total Questions: {total_questions}")
    print(f"Correct Answers: {correct_answers}")
    print(f"Accuracy: {correct_answers / total_questions * 100:.2f}%")


In [ ]:
evaluate_model(llm, test_questions)

### Evaluate Model based on Rouge (Recall-Oriented Understudy for Gissing Evaluation)

A set of metrics used for evaluating the quality of summaries. It compares the generated summary with one or more reference summaries and calculates precision, recall, and F1-score 

- ROUGE-N (quantifies the overlap of N-grams, contiguous sequences of N items (typically words or characters), between the system-generated summary and the reference summary) 
- ROUGE-1 (unigram overlap): This metric measures the overlap of unigrams (single words) between the generated summary and the reference summary. It focuses on the recall of unigrams.
- ROUGE-2 (bigram overlap): Similar to ROUGE-1, but it measures the overlap of bigrams (pairs of consecutive words) instead of unigrams. It evaluates the recall of bigrams.
- ROUGE-L (longest common subsequence): Instead of measuring word overlap, ROUGE-L focuses on the longest common subsequence (LCS) between the generated and reference summaries. It evaluates the recall of the longest common subsequence

- high precision value suggests that the words or phrases churned out by the machine translation or submodel are primarily accurate
- high recall value, ideally close to 1, implies that the content of the machine-generated output aligns closely with the human-made reference. It signifies the model's proficiency in capturing relevant information



In [83]:
#also check if rouge-score library better

#%pip install rouge
from rouge import Rouge

def evaluate_model(model, questions):
    try:
        correct_answers = 0
        total_questions = len(questions)
        model_answers = []
        rouge_scores = {"rouge-1": {"precision": 0, "recall": 0, "f1-score": 0}} 

        for question, expected_answer in questions.items():
            # Get the model's answer for the current question
            prompt = get_prompt(question)
            model_answer = model(prompt)

            # format answer to match properly 
            model_answer = model_answer.rstrip('.')
            #and calculate accuracy (which is currently defined as approximate match where only any word has to match the correct answer.)
            if any(word.lower() in model_answer.lower() for word in expected_answer.split()):
                correct_answers += 1

            # Concatenate model's answer and expected answer into strings for Rouge evaluation
            model_summary = " ".join(model_answer.split())
            reference_summary = " ".join(expected_answer.split())

            # Get Rouge Score
            rouge = Rouge()
            scores = rouge.get_scores(reference_summary,model_summary) ####CHECK IF REF OR MODEL FIRST

            # Check if Rouge calculation was successful - WIP -> see test below 
            if scores:
                scores = scores[0]
                for metric in ["precision", "recall", "f1-score"]:
                    rouge_scores[f"rouge-1"][metric] += scores.get(f"rouge-1", {}).get(metric, 0)

            rouge_score = rouge.get_scores(reference_summary, model_summary)


            # store model answers in array
            model_answers.append({
                "question": question,
                "expected_answer": expected_answer,
                "model_answer": model_answer,
            })

            #print each model answer with score
            print(f"Question: {question}")
            print(f"Expected Answer: {expected_answer}")
            print(f"Model Answer: {model_answer}")
            print(f"Rouge Score: {rouge_score}")
            print("------------")


        # Calculate average Rouge scores
        for metric in ["precision", "recall", "f1-score"]:
            rouge_scores[f"rouge-1"][metric] /= total_questions

        # Print overall evaluation results
        print(f"Total Questions: {total_questions}")
        print(f"Correct Answers: {correct_answers}")
        print(f"Accuracy: {correct_answers / total_questions * 100:.2f}%")
        #print("Rouge Scores:")
        #print(rouge_scores)

        return model_answers, rouge_scores

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [84]:
model_answers, rouge_scores = evaluate_model(llm, test_questions)


llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =       4.79 ms /     4 runs   (    1.20 ms per token,   834.38 tokens per second)
llama_print_timings: prompt eval time =   50879.52 ms /   230 tokens (  221.22 ms per token,     4.52 tokens per second)
llama_print_timings:        eval time =     744.41 ms /     3 runs   (  248.14 ms per token,     4.03 tokens per second)
llama_print_timings:       total time =   51748.80 ms
Llama.generate: prefix-match hit


Question: Write an informal greeting.
Expected Answer: Hey
Model Answer:   Hey!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
------------
Question: Write an informal response to the question: 'How's it going?'
Expected Answer: Hi, going good, hbu?
Model Answer:   Going great!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
------------



llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =       8.03 ms /     6 runs   (    1.34 ms per token,   747.57 tokens per second)
llama_print_timings: prompt eval time =    2553.77 ms /    16 tokens (  159.61 ms per token,     6.27 tokens per second)
llama_print_timings:        eval time =    1248.55 ms /     5 runs   (  249.71 ms per token,     4.00 tokens per second)
llama_print_timings:       total time =    3846.45 ms
Llama.generate: prefix-match hit


Question: Write an informal message inviting someone to dinner tonight
Expected Answer: Hey, wanna join us for dinner tonight?
Model Answer:   "Hey! Want to grab dinner tonight? I'm buying :D"
Rouge Score: [{'rouge-1': {'r': 0.2222222222222222, 'p': 0.2857142857142857, 'f': 0.24999999507812506}, 'rouge-2': {'r': 0.125, 'p': 0.16666666666666666, 'f': 0.14285713795918387}, 'rouge-l': {'r': 0.2222222222222222, 'p': 0.2857142857142857, 'f': 0.24999999507812506}}]
------------



llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =       7.86 ms /    22 runs   (    0.36 ms per token,  2798.98 tokens per second)
llama_print_timings: prompt eval time =    1978.83 ms /    12 tokens (  164.90 ms per token,     6.06 tokens per second)
llama_print_timings:        eval time =    4542.77 ms /    21 runs   (  216.32 ms per token,     4.62 tokens per second)
llama_print_timings:       total time =    6613.51 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =      11.11 ms /    38 runs   (    0.29 ms per token,  3420.03 tokens per second)
llama_print_timings: prompt eval time =    1841.26 ms /    11 tokens (  167.39 ms per token,     5.97 tokens per second)
llama_print_timings:        eval time =    8067.97 ms /    37 runs   (  218.05 ms per token,     4.59 tokens per second)
llama_print_timings:       total time =   10060.87 ms
Llama.generate: prefix-

...


llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =       9.18 ms /    28 runs   (    0.33 ms per token,  3049.11 tokens per second)
llama_print_timings: prompt eval time =    1683.27 ms /    10 tokens (  168.33 ms per token,     5.94 tokens per second)
llama_print_timings:        eval time =    6073.41 ms /    27 runs   (  224.94 ms per token,     4.45 tokens per second)
llama_print_timings:       total time =    7870.14 ms
Llama.generate: prefix-match hit


Question: question
Expected Answer: Extract the dates mentioned in the text 'The conference is scheduled for January 25-27, 2023.'
Model Answer:   Of course! I'm here to help. Please ask your question, and I will do my best to provide a short and simple answer
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
------------



llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =       8.39 ms /    31 runs   (    0.27 ms per token,  3695.76 tokens per second)
llama_print_timings: prompt eval time =     786.80 ms /     5 tokens (  157.36 ms per token,     6.35 tokens per second)
llama_print_timings:        eval time =    6017.23 ms /    30 runs   (  200.57 ms per token,     4.99 tokens per second)
llama_print_timings:       total time =    6920.50 ms
Llama.generate: prefix-match hit


Question: answer
Expected Answer: Dates: January 25-27, 2023.
Model Answer:   Understood! I'll do my best to provide short and simple answers in the language style requested, while being honest, respectful, and socially unbiased. If I don't know the answer to a question, I will say so truthfully without giving false information. Please feel free to ask me anything!
Rouge Score: [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
------------
Total Questions: 20
Correct Answers: 17
Accuracy: 85.00%



llama_print_timings:        load time =   14553.27 ms
llama_print_timings:      sample time =      20.30 ms /    68 runs   (    0.30 ms per token,  3348.93 tokens per second)
llama_print_timings: prompt eval time =     787.52 ms /     5 tokens (  157.50 ms per token,     6.35 tokens per second)
llama_print_timings:        eval time =   14106.35 ms /    67 runs   (  210.54 ms per token,     4.75 tokens per second)
llama_print_timings:       total time =   15164.04 ms


In [ ]:
##test rouge library - https://pypi.org/project/rouge/ 
## -> not clear which rouge metric meant: theres rouge-1/2/l - only referencing paper
##Note: "f" stands for f1_score, "p" stands for precision, "r" stands for recall.


##1 sentence
hypothesis = "I am doing well. How about you?"
reference = "Hello, I am doing well. How about you?"
#reference = "Good day! I'm doing well, thank you for asking. How about you?"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

In [ ]:
##multiple sentences
import json

# Load some sentences
with open('./tests/data.json') as f:
  data = json.load(f)

hyps, refs = map(list, zip(*[[d['hyp'], d['ref']] for d in data]))
rouge = Rouge()
scores = rouge.get_scores(hyps, refs)
# or
scores = rouge.get_scores(hyps, refs, avg=True)


##two files
from rouge import FilesRouge

files_rouge = FilesRouge()
scores = files_rouge.get_scores(hyp_path, ref_path)
# or
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)

In [ ]:
###Correctness Evaluation

In [ ]:
%pip install bitsandbytes
%pip install datasets

In [ ]:
###Performance Evaluation - Perplexity: measure that quantifies how well the model predicts a sample of text. Lower perplexity values indicate better performance 
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer
import bitsandbytes as bnb

model_name = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

tokenizer = LlamaTokenizer.from_pretrained(model_name)

free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map='auto',
  load_in_4bit=True,
  max_memory=max_memory,
  do_sample=True,
  torch_dtype="auto"
)

from datasets import load_dataset

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")


import torch
from tqdm import tqdm

max_length = model.config.max_length
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())